# Estamos em Produção

#### 1 - Carregar o modelo treinado
#### 2 - Carregar os dados a serem inferidos
#### 3 - Fazer as inferências
#### 4 - FIM

In [ ]:
# Carregando modelo

import pickle
model = pickle.load(open('model_GBT.sav', 'rb'))

In [ ]:
import pandas as pd

df = pd.read_excel('O_G_Equipment_Data.xlsx')
df.head(2)

Cycle  Preset_1  Preset_2  Temperature   Pressure  VibrationX  VibrationY  \
0      1         3         6    44.235186  47.657254   46.441769   64.820327   
1      2         2         4    60.807234  63.172076   62.005951   80.714431   

   VibrationZ  Frequency   Fail  
0   66.454520  44.483250  False  
1   81.246405  60.228715  False

In [ ]:
# Codificando a coluna Fail

labels = df['Fail'].astype('category').cat.categories.tolist()
replace_Fail = {'Fail': {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(replace_Fail)

In [ ]:
df.replace(replace_Fail, inplace=True)

In [ ]:
# Trecho de código utilizado para realizar inferências somente com um novo dataset que apresentará as falhas originadas

df_WF = df.copy()

In [ ]:
# Removendo as colunas para realizar a predição 

df = df.drop(columns={'Cycle', 'Temperature',
                          'Pressure',	'VibrationX',
                          'VibrationY','VibrationZ',
                            'Frequency', 'Fail'})
df.head(4)

Preset_1  Preset_2
0         3         6
1         2         4
2         2         1
3         2         3

In [ ]:
# Inferência

inferences = model.predict(df)
df['Resultado'] = inferences
df.head(4)

Preset_1  Preset_2  Resultado
0         3         6   1.081967
1         2         4   1.081967
2         2         1   1.081967
3         2         3   1.081967

In [ ]:
df_t = pd.DataFrame(df.Resultado.value_counts().sort_index())
df_t = df_t.reset_index()
df_t = df_t.rename({'index': 'Fail_Category'}, axis=1)
df_t

## CATEGORIZANDO AS FALHAS, CRIANDO UM GRAU DE INTENSIDADE E/OU CRITICIDADE PARA A FALHA DETECTADA.

In [ ]:
#df_WF = df.copy()
df_WF.drop(df_WF[(df_WF['Fail'] == 1)].index, inplace=True)
df_WF.head(4)

Cycle  Preset_1  Preset_2  Temperature    Pressure  VibrationX  \
12      13         1         1    79.806509  103.129738  120.753103   
156    157         3         1    91.695332   85.345796   99.890670   
157    158         2         5    88.916114   88.231887   85.140550   
158    159         3         5    88.768766   99.228691   89.109514   

     VibrationY  VibrationZ   Frequency  Fail  
12   134.500905  115.873197   76.562856     2  
156   95.328429  124.970301   84.377355     2  
157  123.711911   88.852872  100.229131     2  
158  124.934221   84.306467   88.218162     2

In [ ]:
df_2 = df_WF.drop(columns={'Cycle', 'Temperature',
                          'Pressure',	'VibrationX',
                          'VibrationY','VibrationZ',
                            'Frequency', 'Fail'})
df_2.head(4)

Preset_1  Preset_2
12          1         1
156         3         1
157         2         5
158         3         5

In [ ]:
# Inferência

inferences = model.predict(df_2)
df_2['Resultado'] = inferences
df_2.head(4)

Preset_1  Preset_2  Resultado
12          1         1   1.110465
156         3         1   1.081967
157         2         5   1.081967
158         3         5   1.081967

In [ ]:
df_f = pd.DataFrame(df_2.Resultado.value_counts().sort_index())

df_f = df_f.reset_index()

df_f = df_f.rename({'index': 'Fail_Category'}, axis=1)
df_f

Fail_Category  Resultado
0       1.046296         12
1       1.072727          4
2       1.081967         29
3       1.110465         21